## Analyze e2e latencies for the single_client_e2e experiment

In [26]:
import re
import numpy as np
computee2e, confirme2e = [], []
consumee2e = []
avg_batching_latencies = []
avg_batch_size = []
with open("../logs/client_node7.log", "r") as f:
    lines = f.readlines()
    for line in lines:
        if "latencies: " in line:
            consume, confirm, compute = line.strip().split('latencies: ')[-1].split(',')[-3:]
            consumee2e.append(int(consume))
            confirme2e.append(int(confirm))
            computee2e.append(int(compute))
        if "average batch size: " in line:
            avg_batch_size.append(float(line.strip().split()[-1]))
        if "average batching latency: " in line:
            avg_batching_latencies.append(float(line.strip().split()[-1]))

computee2e = np.array(computee2e, dtype=int)
confirme2e = np.array(confirme2e, dtype=int)
consumee2e = np.array(consumee2e, dtype=int)

temp = np.maximum(computee2e, confirme2e)

print("statistic/metric, compute e2e (us), confirm e2e (us), consume e2e (us), total e2e (us)")
print(f"mean, {np.mean(computee2e):.2f}, {np.mean(confirme2e):.2f}, {np.mean(consumee2e):.2f}, {np.mean(temp):.2f}")
print(f"std, {np.std(computee2e):.2f}, {np.std(confirme2e):.2f}, {np.std(consumee2e):.2f}, {np.std(temp):.2f}")
print(f"p50, {np.percentile(computee2e, 50):.2f}, {np.percentile(confirme2e, 50):.2f}, {np.percentile(consumee2e, 50):.2f}, {np.percentile(temp, 50):.2f}")
print(f"p99, {np.percentile(computee2e, 90):.2f}, {np.percentile(confirme2e, 90):.2f}, {np.percentile(consumee2e, 90):.2f}, {np.percentile(temp, 90):.2f}")

print("average batch size, average batching latencies")
print(f"{avg_batch_size[-1]:.2f}, {avg_batching_latencies[-1]:.2f}")



statistic/metric, compute e2e (us), confirm e2e (us), consume e2e (us), total e2e (us)
mean, 1807.20, 2633.06, 787.75, 2651.66
std, 567.27, 542.00, 261.30, 526.72
p50, 1855.00, 2815.00, 739.00, 2815.00
p99, 2445.00, 3164.00, 920.00, 3164.00
average batch size, average batching latencies
1.58, 30.41
